## Matrix and Vocabulary Construction

In [29]:
#coding: utf-8

import pandas as pd

import numpy as np

from scipy import sparse

from operator import itemgetter

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

In [43]:
news = pd.read_csv("estadao_noticias_eleicao.csv", encoding="utf-8")

In [44]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

In [45]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

#### Removing punctuation

In [46]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

#### Removing stopwords

In [47]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

#### Transforming list of lists into one list

In [48]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [49]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consult Bigram Frequency

In [50]:
consultable_matrix = matrix.tocsr()

In [51]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

def get_coocurrrency(word):
    results = consultable_matrix[vocab[word]].toarray()[0]
    indexes, L_sorted = zip(*sorted(enumerate(results), key=itemgetter(1),reverse=True))
    print indexes[:5]
    print L_sorted[:5]
    return [ str(vocab.keys()[vocab.values().index(index)]) for index in indexes[:5] ]

#### Example

In [54]:
w1 = 'poucos'
w2 = 'recursos'
consult_frequency(w1, w2)
get_coocurrrency("petrobrás")

KeyError: 'petrobr\xc3\xa1s'